<a href="https://colab.research.google.com/github/TeinkBR/intento_task/blob/master/additional_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas --quiet
!pip install nltk --quiet
!pip install tmx2dataframe --quiet

  Preparing metadata (setup.py) ... done


In [8]:
!pip install xml --quiet

ERROR: Could not find a version that satisfies the requirement xml (from versions: none)
ERROR: No matching distribution found for xml


In [35]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from tmx2dataframe import tmx2dataframe
import xml.parsers.expat
class DatasetCleaner:
    """
    A class for cleaning and filtering bilingual datasets stored in TMX files.
    """

    def __init__(self, file_name):
        """
        Initialize the DatasetCleaner object with the given TMX file name.
        """
        self.file_name = file_name
        self.df = self.read_tmx_file()
    
    def read_tmx_file(self):
      """
      Read the TMX file and return a Pandas DataFrame.
      """
      metadata, df = tmx2dataframe.read(self.file_name)
      df = df.rename(columns={"source_sentence": "source", "target_sentence": "target"})
      df['inner_id'] = range(1, len(df) + 1)
      return df


    '''
    def read_tmx_files(self):
        """
        Read the TMX files and return a Pandas DataFrame.
        """
        segments = []
        for file_name in self.file_names:
            tree = ET.parse(file_name)
            root = tree.getroot()

            for tu in root.iter('tu'):
                source_text = ""
                target_text = ""

                for tuv in tu.iter('tuv'):
                    lang = tuv.attrib['xml:lang']
                    seg = tuv.find('seg').text

                    if lang == 'en':
                        source_text = seg
                    elif lang == 'pt':
                        target_text = seg

                segments.append((source_text, target_text))

        df = pd.DataFrame(segments, columns=['source', 'target'])
        df['inner_id'] = range(1, len(df) + 1)
        return df
    
    
    def read_tmx_files(self):
      """
      Read the TMX files and return a Pandas DataFrame.
      """
      segments = []

      for file_name in self.file_names:
          try:
              _, df_temp = tmx2dataframe.read(file_name)
              segments += list(zip(df_temp['en'], df_temp['pt']))
          except xml.parsers.expat.ExpatError as e:
              print(f"Error parsing {file_name}: {e}")
              continue

      df = pd.DataFrame(segments, columns=['source', 'target'])
      df['inner_id'] = range(1, len(df) + 1)
      return df
    '''
    def delete_duplicates(self):
        """
        Delete duplicates in source, target, and source+target.
        """
        self.df.drop_duplicates(subset=['source'], inplace=True)
        self.df.drop_duplicates(subset=['target'], inplace=True)
        self.df.drop_duplicates(subset=['source', 'target'], inplace=True)

    def token_count(self, text, lang):
        """
        Return the number of tokens in the given text using NLTK.
        """
        return len(word_tokenize(text, language=lang))

    def symbol_count(self, text):
        """
        Return the number of symbols in the given text.
        """
        return len(text)

    def sentence_count(self, text, lang):
        """
        Return the number of sentences in the given text using NLTK.
        """
        return len(sent_tokenize(text, language=lang))

    def has_link(self, text):
        """
        Check if the text contains a link using a regular expression.
        """
        return bool(
            re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text))

    def clean_dataset(self):
        """
        Clean the dataset according to the given requirements.
        """
        original_length = len(self.df)
        print(f"Original dataset length: {original_length}")
        

        self.delete_duplicates()
        self.apply_filters()

        self.df_filtered = self.df[
            (self.df['source_token_count'] >= 4) & (self.df['source_token_count'] <= 200) &
            (self.df['target_token_count'] >= 4) & (self.df['target_token_count'] <= 200) &
            (self.df['source_symbol_count'] >= 15) & (self.df['source_symbol_count'] <= 450) &
            (self.df['target_symbol_count'] >= 15) & (self.df['target_symbol_count'] <= 450) &
            (self.df['source_sentence_count'] == 1) & (self.df['target_sentence_count'] == 1) &
            (~self.df['has_link']) &
            (~self.df['digit_mismatch']) &
            (~self.df['invalid_target_start']) &
            (~self.df['invalid_source_middle'])
            ]
        print(f"Filtered dataset length: {len(self.df_filtered)}")
        filtered_length = len(self.df_filtered)
        print(f"Filtered dataset length: {filtered_length}")
        print(f"Number of rows removed: {original_length - filtered_length}")
    def apply_filters(self):
      """
      Apply various filters to the dataset.
      """
      self.df['source_token_count'] = self.df['source'].apply(self.token_count, lang='english')
      self.df['target_token_count'] = self.df['target'].apply(self.token_count, lang='polish')
      self.df['source_symbol_count'] = self.df['source'].apply(self.symbol_count)
      self.df['target_symbol_count'] = self.df['target'].apply(self.symbol_count)
      self.df['source_sentence_count'] = self.df['source'].apply(self.sentence_count, lang='english')
      self.df['target_sentence_count'] = self.df['target'].apply(self.sentence_count, lang='polish')
      self.df['has_link'] = self.df['source'].apply(self.has_link) | self.df['target'].apply(self.has_link)

      # Use a Pandas Series instead of DataFrame for 'digit_mismatch'
      self.df['digit_mismatch'] = self.df.apply(
          lambda row: set(re.findall(r'\d+', row['source'])) != set(re.findall(r'\d+', row['target'])), axis=1)

      self.df['invalid_target_start'] = self.df['target'].str.startswith('Então')
      self.df['invalid_source_middle'] = self.df['source'].apply(lambda x: 'actually' in x.split()[1:-1])


    def summarize(self):
        """
        Summarize the dataset by showing min and max inner_id, mean and median of source segment length in tokens and symbols.
        """
        min_inner_id = self.df_filtered['inner_id'].min()
        max_inner_id = self.df_filtered['inner_id'].max()
        mean_source_token_count = self.df_filtered['source_token_count'].mean()
        median_source_token_count = self.df_filtered['source_token_count'].median()
        mean_source_symbol_count = self.df_filtered['source_symbol_count'].mean()
        median_source_symbol_count = self.df_filtered['source_symbol_count'].median()

        summary = {
            'min_inner_id': min_inner_id,
            'max_inner_id': max_inner_id,
            'mean_source_token_count': mean_source_token_count,
            'median_source_token_count': median_source_token_count,
            'mean_source_symbol_count': mean_source_symbol_count,
            'median_source_symbol_count': median_source_symbol_count
        }

        return summary

    def save_to_excel(self, file_name):
        """
        Save the DataFrame to an Excel file.
        """
        self.df_filtered.to_excel(file_name, index=False)
    def save_to_csv(self, file_name):
      """
      Save the cleaned DataFrame to a CSV file.
      """
      self.df_filtered.to_csv(file_name, index=False)
    
    
    def get_cleaned_dataframe(self):
        """
        Return the cleaned DataFrame.
        """
        return self.df_filtered
    

"""
if __name__ == '__main__':
    file_names = ['hr-it.tmx', 'es-is.tmx', 'en-pl.tmx']
    cleaned_file_names = ['Cleaned_hr-it_dataset.csv', 'Cleaned_es-is_dataset.csv', 'Cleaned_en-pl_dataset.csv']

    for i, file_name in enumerate(file_names):
        dataset_cleaner = DatasetCleaner([file_name])
        dataset_cleaner.clean_dataset()
        dataset_cleaner.df_filtered.sort_values(by=['source_symbol_count'], ascending=True, inplace=True)
        summary = dataset_cleaner.summarize()
        print(f"Summary for {file_name}: {summary}")
        dataset_cleaner.df_filtered.to_csv(cleaned_file_names[i], index=False)
"""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'\nif __name__ == \'__main__\':\n    file_names = [\'hr-it.tmx\', \'es-is.tmx\', \'en-pl.tmx\']\n    cleaned_file_names = [\'Cleaned_hr-it_dataset.csv\', \'Cleaned_es-is_dataset.csv\', \'Cleaned_en-pl_dataset.csv\']\n\n    for i, file_name in enumerate(file_names):\n        dataset_cleaner = DatasetCleaner([file_name])\n        dataset_cleaner.clean_dataset()\n        dataset_cleaner.df_filtered.sort_values(by=[\'source_symbol_count\'], ascending=True, inplace=True)\n        summary = dataset_cleaner.summarize()\n        print(f"Summary for {file_name}: {summary}")\n        dataset_cleaner.df_filtered.to_csv(cleaned_file_names[i], index=False)\n'

In [36]:
if __name__ == '__main__':
    file_name = 'en-pl.tmx'
    dataset_cleaner = DatasetCleaner(file_name)
    dataset_cleaner.clean_dataset()
    dataset_cleaner.df_filtered.sort_values(by=['source_symbol_count'], ascending=True, inplace=True)
    summary = dataset_cleaner.summarize()
    print(summary)
    dataset_cleaner.save_to_csv('Cleaned_EN_PL_dataset.csv')

Original dataset length: 148048
Filtered dataset length: 109565
Filtered dataset length: 109565
Number of rows removed: 38483
{'min_inner_id': 3, 'max_inner_id': 148046, 'mean_source_token_count': 18.02450600100397, 'median_source_token_count': 15.0, 'mean_source_symbol_count': 83.89812440104048, 'median_source_symbol_count': 69.0}


<ipython-input-36-ea35e6a77814>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cleaner.df_filtered.sort_values(by=['source_symbol_count'], ascending=True, inplace=True)


In [28]:
dataset_cleaner = DatasetCleaner('en-pl.tmx')
dataset_cleaner.clean_dataset()
cleaned_df = dataset_cleaner.get_cleaned_dataframe()
cleaned_df.head()


,source_language,source,target_language,target,inner_id,source_token_count,target_token_count,source_symbol_count,target_symbol_count,source_sentence_count,target_sentence_count,has_link,digit_mismatch,invalid_target_start,invalid_source_middle
2,en,"fish,health,mission blue,oceans,science",pl,"fish,health,mission blue,oceans,science",3,10,10,39,39,1,1,False,False,False,False
4,en,Stephen Palumbi: Following the mercury trail,pl,Stephen Palumbi: Podążając rtęciowym szlakiem,5,7,6,44,45,1,1,False,False,False,False
5,en,"It can be a very complicated thing, the ocean.",pl,Ocean jest skomplikowany.,6,11,4,46,25,1,1,False,False,False,False
6,en,"And it can be a very complicated thing, what h...",pl,Ludzkie zdrowie również.,7,14,4,61,24,1,1,False,False,False,False
8,en,And those simple themes aren't really themes a...,pl,"Nie są to ściśle naukowe rzeczy, Nie są to ści...",9,27,21,131,98,1,1,False,False,False,False


In [41]:
print(pd.DataFrame(cleaned_df))

       source_language                                             source  \
2                   en            fish,health,mission blue,oceans,science   
4                   en       Stephen Palumbi: Following the mercury trail   
5                   en     It can be a very complicated thing, the ocean.   
6                   en  And it can be a very complicated thing, what h...   
8                   en  And those simple themes aren't really themes a...   
...                ...                                                ...   
148039              en                  I would argue life is a rheostat.   
148042              en  Wikipedia, every day, is tens of thousands of ...   
148043              en  It's a perfect image for the fundamental point...   
148044              en  No one person, no one alliance, no one nation,...   
148045              en  The vision statement of Wikipedia is very simp...   

       target_language                                             target  

In [37]:
pip install tensorflow --quiet

In [38]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Load the cleaned data
cleaned_data = pd.read_csv('Cleaned_EN_PL_dataset.csv')


In [42]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the source and target sentences
source_tokenizer = Tokenizer(filters='')
source_tokenizer.fit_on_texts(cleaned_data['source'])
source_sequences = source_tokenizer.texts_to_sequences(cleaned_data['source'])

target_tokenizer = Tokenizer(filters='')
target_tokenizer.fit_on_texts(cleaned_data['target'])
target_sequences = target_tokenizer.texts_to_sequences(cleaned_data['target'])

# Pad the sequences
max_source_seq_length = max([len(seq) for seq in source_sequences])
max_target_seq_length = max([len(seq) for seq in target_sequences])

source_data = pad_sequences(source_sequences, maxlen=max_source_seq_length, padding='post')
target_data = pad_sequences(target_sequences, maxlen=max_target_seq_length, padding='post')


In [43]:
source_train, source_val, target_train, target_val = train_test_split(source_data, target_data, test_size=0.2, random_state=42)

In [45]:
from tensorflow.keras.layers import Embedding

In [46]:
embedding_dim = 256
latent_dim = 1024

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(len(source_tokenizer.word_index) + 1, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(len(target_tokenizer.word_index) + 1, embedding_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

decoder_dense = Dense(len(target_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

batch_size = 64
epochs = 30
history = model.fit([source_train, target_train], np.expand_dims(target_train, -1),
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=([source_val, target_val], np.expand_dims(target_val, -1)))


Epoch 1/30
   1/1370 [..............................] - ETA: 62:49:34 - loss: 12.0265